# Project2_2

Notebook to design and test a Kalman filter. Different solutions for 3, 4 and 6 states are available in addition to an extented Kalman filter.

In [ ]:
# Insérez ici le numéro de votre équipe
# (en) Insert here your team number
VOTRE_NUMERO_EQUIPE = 0

In [ ]:
import sys
sys.path.append('/opt/ros/noetic/lib/python3/dist-packages/')

In [ ]:
# Import des paquets requis et configuration du ROS Master
# (en) Import of required packages and configuration of the ROS Master
from os import environ
import numpy as np
import rospy
from scipy.spatial.transform import Rotation as R
from math import sin, cos
from jackal_msgs.msg import Feedback
from jackal_msgs.msg import Drive
from sensor_msgs.msg import Imu
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Point, Pose, Quaternion, Twist, Vector3
from filterpy.kalman import KalmanFilter
#from lab_utils.project2 import publish_odom_tf

In [ ]:
environ['ROS_MASTER_URI'] = "http://localhost:1137{}/".format(VOTRE_NUMERO_EQUIPE)

In [ ]:
# Créer et démarrer un nouveau noeud
# (en) Create and start a new node
rospy.init_node('dingo_state_estimation', anonymous=True)

In [ ]:
# Fonction pour le calcul de l'orientation à partir d'un quaternion
# Entrée : Quaternion [x, y ,z ,w]
# Sortie : Angle de lacet (yaw) en radians
# (en) Function for calculating the orientation from a quaternion
# Input: Quaternion [x, y ,z ,w]
# Output: Yaw angle in radians
def get_heading_from_quaternion(q):
    r = R.from_quat([q.x, q.y, q.z, q.w])
    angles = r.as_euler('xyz', degrees=False)
    return angles[2]

In [ ]:
# Ground truth subscriber callback
# (en) Ground truth subscriber callback
ground_truth_msg = Odometry()
def ground_truth_callback(msg):
    global ground_truth_msg
    ground_truth_msg = msg

In [ ]:
# UMI subscriber callback
# Dans cette fonction la variable msg contient les mesures de l'UMI
# (en) UMI subscriber callback
# In this function the variable msg contains the measurements of the UMI
def imu_callback(msg):
    # Utiliser les valeurs du IMU dans cette fonction
    # (en) Use the values of the IMU in this function

    # Ajouter z et Timestamp
    # (en) Add z and Timestamp
    kf_imu_update(z, timestamp) # Optionnel selon le filtre choisi

### Configuration du script (Script configuration)

Ajoutez ici les données géométriques du robot.

(en) Add here the geometric data of the robot.

In [ ]:
# À compléter selon les tâches précédentes
# (en) To be completed according to the previous tasks
interwheel_distance = 0.0
left_wheel_radius = 0.0
right_wheel_radius = 0.0

### Estimer les déplacements du robot (Estimate the robot's displacements)

Dans la prochaine cellule ajoutez votre code pour estimer les variables x, y et theta du robot (et leurs vitesses).

(en) In the next cell add your code to estimate the variables x, y and theta of the robot (and their speeds).

In [ ]:
# Encodeurs subscriber callback
# (en) Encoders subscriber callback

# cette fonction se charge aussi de calculer l'odométrie pour mettre à jour l'FK
# l'utilisation de ces variables est suggérée
# (en) This function also takes care of calculating the odometry to update the FK
# the use of these variables is suggested
prev_left_pos = 0.0
prev_right_pos = 0.0
last_time_ = rospy.Time.now()
def encoders_callback(msg):
    global prev_left_pos, prev_right_pos, last_time_

    # Utiliser les valeurs des encodeurs dans cette fonction
    # (en) Use the values of the encoders in this function

    # Ajouter z et Timestamp
    # (en) Add z and Timestamp
    kf_odom_update(z, timestamp) # Optionnel selon le filtre choisi

### Construire le filtre (Build the filter)

Dans la prochaine cellule utiliser la bibliothèque FilterPy pour construire le filtre Kalman et estimer le déplacement du robot à partir des capteurs.

(en) In the next cell use the FilterPy library to build the Kalman filter and estimate the robot's displacement from the sensors.

Vous devez concevoir votre filtre et prendre en compte plusieurs éléments par exemple : nombre d'états, nombre de commandes, nombre de mesures, synchrone ou asynchrone, KF ou EKF, etc.

(en) You must design your filter and take into account several elements for example: number of states, number of commands, number of measurements, synchronous or asynchronous, KF or EKF, etc.

In [ ]:
# Création du filtre Kalman - dimensions x et z initialisées (changez le 0 selon votre utilisation)
# (en) Creation of the Kalman filter - dimensions x and z initialized (change the 0 according to your use)
kf = KalmanFilter(dim_x=0, dim_z=0)

# Fréquence de mise à jour (changez le 1 pour la fréquence des msgs en entrée)
# (en) Update frequency (change 1 for the frequency of the input msgs)
dt = 1.0

# État de départ (changez le 0 selon votre utilisation)
# (en) Starting state (change 0 according to your use)
kf.x = np.zeros(0)

# Matrice de bruit sur le modèle
# Pour le fonctionnement de la librairie, kf.Q = B*Q*B^T
# (en) Noise matrix on the model
kf.Q = np.array([# entrez ici une matrix de la forme [ligne1],[ligne2]
                 # (en) enter here a matrix of the form [line1],[line2]
                ])

# Matrice de transition d'états
# (en) State transition matrix
kf.F = np.array([# entrez ici une matrix de la forme [ligne1],[ligne2]
                 # (en) enter here a matrix of the form [line1],[line2]
                ])

# Matrice de bruit sur l'odométrie (selon votre utilisation)
# (en) Noise matrix on odometry (according to your use)
kf.R_odom = np.array([# entrez ici une matrix de la forme [ligne1],[ligne2]
                      # (en) enter here a matrix of the form [line1],[line2]
                        ])

# Matrice de transition odométrie-états (selon votre utilisation)
# (en) Odometry-state transition matrix (according to your use)
kf.H_odom = np.array([# entrez ici une matrix de la forme [ligne1],[ligne2]
                      # (en) enter here a matrix of the form [line1],[line2]
                        ])

# Matrice de bruit sur l'UMI (selon votre utilisation)
# (en) Noise matrix on the IMU (according to your use)
kf.R_imu = np.array([# entrez ici une matrix de la forme [ligne1],[ligne2]
                     # (en) enter here a matrix of the form [line1],[line2]
                    ])

# Matrice de transition UMI-états (selon votre utilisation)
# (en) IMU-state transition matrix (according to your use)
kf.H_imu =  np.array([# entrez ici une matrix de la forme [ligne1],[ligne2]
                      # (en) enter here a matrix of the form [line1],[line2]
                    ])

last_measurement_time = 0
def kf_pre_update(timestamp):
    global last_measurement_time

    if timestamp > last_measurement_time:
        dt = timestamp - last_measurement_time
        last_measurement_time = timestamp

        # v = kf_cmd_vector[0] # Optionnel selon le filtre choisi
        # (en) # Optional depending on the filter used

        # Mettre à jour les matrices qui dépendent du temps si on utilise un EKF
        # (en) Update matrices that depend on time if using an EKF


        # À remplir au besoin
        # (en) To be filled in as needed

    else:
        pass

    kf.predict(u=kf_cmd_vector)


def kf_odom_update(z, timestamp): # Optionnel selon le filtre choisi
    kf_pre_update(timestamp)

    kf.H = kf.H_odom

    kf.R = kf.R_odom

    kf.update(z)

def kf_imu_update(z, timestamp): # Optionnel selon le filtre choisi
    kf_pre_update(timestamp)

    kf.H = kf.H_imu

    kf.R = kf.R_imu

    kf.update(z)

### Configuration du script (Script configuration)

Ajoutez ici les noms des messages ROS.

(en) Add here the names of the ROS messages.

In [ ]:
# ROS subscribers et publishers
# (en) ROS subscribers and publishers
feedback_sub = rospy.Subscriber('XXX', Feedback, encoders_callback)
imu_sub = rospy.Subscriber('XXX', Imu, imu_callback)
ground_truth_sub = rospy.Subscriber('XXX', Odometry, ground_truth_callback)
cmd_drive_pub = rospy.Publisher('XXX', Drive, queue_size=1)

### Tester le filtre (Test the filter)

La prochaine cellule déplace le robot dans une courbe pour 20 secondes et enregistre sa pose réelle la pose estimée dans un rosbag.

(en) The next cell moves the robot in a curve for 20 seconds and records its actual pose the estimated pose in a rosbag.

Changez les 0 pour les valeurs correspondantes de votre filtre Kalman.

(en) Change the 0 for the corresponding values of your Kalman filter.

In [ ]:
# Création d'un nouveau contenant ROSBAG (en écriture)
# (en) Creation of a new ROSBAG container (in writing)
import rosbag
test_bag = rosbag.Bag('project2.bag', 'w')

# Publie l'odométrie et la TF pendant 20 secondes
# (en) Publish odometry and TF for 20 seconds
start = float(rospy.Time().now().secs)
rate = rospy.Rate(50) # 50hz
while (float(rospy.Time().now().secs) - start) < 20:

    # ***TÂCHE***
    # Remplissez avec les valeurs correspondantes de votre filtre Kalman (états)
    # (en) Fill in with the corresponding values of your Kalman filter (states)
    # exemple x = kf.x[0]
    x =
    y =
    vlin =
    vang =
    theta =
    # ***TÂCHE***

    # Déplacer le robot
    # (en) Move the robot
    cmd_drive_msg = Drive()
    cmd_drive_msg.drivers[0] = 5.0
    cmd_drive_msg.drivers[1] = 7.0
    cmd_drive_pub.publish(cmd_drive_msg)

    # Enregistrer le déplacement du robot
    # (en) Record the robot's movement
    pose = (x, y, 0)
    r = R.from_euler('xyz', [0, 0, theta], degrees=False)
    orientation = r.as_quat()
    odometry_msg = Odometry()
    odometry_msg.header.frame_id = "odom"
    odometry_msg.header.stamp = rospy.Time.now()
    odometry_msg.child_frame_id = "base_link"
    odometry_msg.pose.pose = Pose(Point(*pose), Quaternion(*orientation))
    odometry_msg.twist.twist = Twist(Vector3(vlin, 0, 0), Vector3(0, 0, vang))

    test_bag.write('/filter', odometry_msg, rospy.Time().now())
    test_bag.write('/ground_truth', ground_truth_msg, rospy.Time().now())
    rate.sleep()

# fermer proprement le ROSBAG
# (en) close the ROSBAG properly
test_bag.close()

In [ ]:
# Arreter le dingo
# (en) Stop the dingo
cmd_drive_msg = Drive()
cmd_drive_msg.drivers[0] = 0.0
cmd_drive_msg.drivers[1] = 0.0
cmd_drive_pub.publish(cmd_drive_msg)